In [1]:
from repalette.lightning.datamodules import AdversarialDataModule
from repalette.lightning.systems import PreTrainSystem, AdversarialSystem
from repalette.lightning.callbacks import Notify

from repalette.constants import S3_PRETRAINED_MODEL_CHECKPOINT_PATH, LIGHTNING_LOGS_DIR, MODEL_CHECKPOINTS_DIR

In [2]:
!source "../.env"

In [3]:
pretrain_system = PreTrainSystem.load_from_checkpoint("/home/dan/Projects/repalette/data/model-checkpoints/final/epoch=4-step=494599.ckpt")

In [4]:
adversarial_datamodule = AdversarialDataModule(multiplier=16, size=0.1, batch_size=8, num_workers=14)

In [5]:
import pytorch_lightning as pl

In [6]:
from repalette.lightning.callbacks import LogTripletRecoloringToTensorboard
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, GPUStatsMonitor
import uuid
import os

log_recoloring_to_tensorboard = LogTripletRecoloringToTensorboard(batches=3)

logger = TensorBoardLogger(
    LIGHTNING_LOGS_DIR,
    name="final_train",
    log_graph=False,
    default_hp_metric=False,
    )

adv_checkpoints = ModelCheckpoint(
        dirpath=os.path.join(MODEL_CHECKPOINTS_DIR, str(logger.version)),
        monitor="Val/generator_loss_epoch",
        verbose=True,
        mode="min",
        save_top_k=-1,
    )

In [7]:
trainer = pl.Trainer(gpus=-1,
     logger=logger,
     benchmark=True,
    enable_pl_optimizer=True,
     callbacks=[
         log_recoloring_to_tensorboard,
     ]
    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.metrics.regression import MeanSquaredError
from torch import nn as nn

from repalette.constants import (
    DEFAULT_DISCRIMINATOR_LR,
    DEFAULT_GENERATOR_LR,
    DEFAULT_ADVERSARIAL_BETA_1,
    DEFAULT_ADVERSARIAL_BETA_2,
    DEFAULT_GENERATOR_WEIGHT_DECAY,
    DEFAULT_ADVERSARIAL_LAMBDA_MSE_LOSS,
)
from repalette.models import PaletteNet, Discriminator

In [9]:
adversarial_system = AdversarialSystem(
    generator_learning_rate=DEFAULT_GENERATOR_LR, generator=pretrain_system.generator, k=10, discriminator_learning_rate=DEFAULT_DISCRIMINATOR_LR, discriminator_weight_decay=0.01
)

In [ ]:
trainer.fit(adversarial_system, adversarial_datamodule)


  | Name          | Type          | Params
------------------------------------------------
0 | generator     | PaletteNet    | 13.9 M
1 | discriminator | Discriminator | 2.8 M 
2 | MSE           | MSELoss       | 0     
------------------------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]